In [1]:
from clickhouse_driver import Client

# ClickHouse connection parameters
CLICKHOUSE_HOST = "localhost"
CLICKHOUSE_PORT = 9000
CLICKHOUSE_USER = "cs2_user"
CLICKHOUSE_PASSWORD = "cs2_password"
CLICKHOUSE_DB = "cs2_db"


# Create ClickHouse client
client = Client(
    host=CLICKHOUSE_HOST,
    port=CLICKHOUSE_PORT,
    user=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD,
    database=CLICKHOUSE_DB
)




In [2]:
def fetch_games_by_ids(client: Client, game_ids, clickhouse_db="cs2_db"):
    if len(game_ids) == 0:
        return np.array([]), np.array([])
    
    ids_str = ",".join(str(i) for i in game_ids)

    query = f"""
    SELECT
        g.game_id,
        any(g.begin_at) AS timestamp,
        any(g.map_id) AS map_id,
        any(g.league_id) AS league_id,
        any(g.serie_id) AS serie_id,
        any(g.tournament_id) AS tournament_id,
        any(g.tier_id) AS tier_id,
        t.teams[1] AS t1_id,
        t.teams[2] AS t2_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 1) AS p1_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 2) AS p2_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 3) AS p3_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 4) AS p4_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 5) AS p5_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 1) AS p6_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 2) AS p7_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 3) AS p8_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 4) AS p9_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 5) AS p10_id,
        sumIf(g.round_win, g.team_id = t.teams[1]) > sumIf(g.round_win, g.team_id = t.teams[2]) AS team1_win
    FROM {clickhouse_db}.games_flatten AS g
    INNER JOIN (
        SELECT
            game_id,
            arraySort(arrayDistinct(groupArray(team_id))) AS teams
        FROM {clickhouse_db}.games_flatten
        WHERE game_id IN ({ids_str})
        GROUP BY game_id
    ) AS t USING (game_id)
    GROUP BY g.game_id, t.teams
    ORDER BY timestamp ASC
    """

    result = client.query_dataframe(query)
    return result
    if not result:
        return np.array([]), np.array([])

    rows = [list(r) for r in result[0]]

    for row in rows:
        if hasattr(row[1], "timestamp"):
            row[1] = row[1].timestamp()

    X = np.array([r[1:-1] for r in rows], dtype=float)
    y = np.array([r[-1] for r in rows], dtype=int)

    return X, y

In [50]:
df_train

,game_id,timestamp,map_id,league_id,serie_id,tournament_id,tier_id,t1_id,t2_id,p1_id,p2_id,p3_id,p4_id,p5_id,p6_id,p7_id,p8_id,p9_id,p10_id,team1_win
0,1744,2016-04-13 16:30:00,8,4569,1544,1583,1,3216,3224,17539,17540,17542,17570,17602,17529,17576,17578,17579,17588,1
1,2365,2016-06-11 03:01:00,8,4154,1559,1714,0,3209,3224,17507,17508,17509,17510,17524,17529,17576,17578,17579,17655,0
2,1287,2016-09-08 13:30:00,8,4165,1591,1574,0,3209,3216,17507,17508,17509,17510,17524,17525,17539,17540,17541,17542,1
3,2011,2016-12-10 00:40:00,4,4153,1552,1567,0,3222,3245,17513,17515,17516,17568,17708,17501,17581,17658,17664,17665,1
4,4436,2017-03-01 21:00:00,3,4161,1583,1642,0,3212,3245,17522,17523,17524,17565,17569,17501,17543,17581,17664,17665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37026,70114,2022-04-13 10:15:39,2,4163,4541,7883,2,126709,129501,20569,25589,27555,28854,34000,17499,17747,21433,21439,25439,0
37027,70112,2022-04-13 10:18:37,8,4163,4541,7883,2,125802,126377,20678,20684,20692,23776,29284,17497,17498,17501,17543,19666,0
37028,70113,2022-04-13 11:29:14,2,4163,4541,7883,2,125802,126377,20678,20684,20692,23776,29284,17497,17498,17501,17543,19666,0
37029,70115,2022-04-13 11:30:45,7,4163,4541,7883,2,126709,129501,20569,25589,27555,28854,34000,17499,17747,21433,21439,25439,1


In [ ]:
from clickhouse_driver import Client
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# -----------------------------
# ClickHouse connection
# -----------------------------
client = Client(
    host="localhost",
    port=9000,
    user="cs2_user",
    password="cs2_password",
    database="cs2_db"
)

# -----------------------------
# Fetch games by IDs
# -----------------------------
def fetch_games_by_ids(client: Client, game_ids, clickhouse_db="cs2_db"):
    if len(game_ids) == 0:
        return pd.DataFrame()
    
    ids_str = ",".join(str(i) for i in game_ids)
    query = f"""
    SELECT
        g.game_id,
        any(g.begin_at) AS timestamp,
        any(g.map_id) AS map_id,
        any(g.league_id) AS league_id,
        any(g.serie_id) AS serie_id,
        any(g.tournament_id) AS tournament_id,
        any(g.tier_id) AS tier_id,
        t.teams[1] AS t1_id,
        t.teams[2] AS t2_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 1) AS p1_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 2) AS p2_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 3) AS p3_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 4) AS p4_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 5) AS p5_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 1) AS p6_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 2) AS p7_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 3) AS p8_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 4) AS p9_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 5) AS p10_id,
        sumIf(g.round_win, g.team_id = t.teams[1]) > sumIf(g.round_win, g.team_id = t.teams[2]) AS team1_win
    FROM {clickhouse_db}.games_flatten AS g
    INNER JOIN (
        SELECT
            game_id,
            arraySort(arrayDistinct(groupArray(team_id))) AS teams
        FROM {clickhouse_db}.games_flatten
        WHERE game_id IN ({ids_str})
        GROUP BY game_id
    ) AS t USING (game_id)
    GROUP BY g.game_id, t.teams
    ORDER BY timestamp ASC
    """
    df = client.query_dataframe(query)
    return df

# -----------------------------
# Initialize cumulative columns
# -----------------------------
def initialize_cumulative_columns(df: pd.DataFrame, prefix="p", stat="kills", n_players=10):
    cols = [f"{prefix}{i+1}_{stat}_sum" for i in range(n_players)]
    for col in cols:
        df[col] = 0
    return df, cols

# -----------------------------
# Fill cumulative stat
# -----------------------------
def fill_cumulative_stat(client: Client, df: pd.DataFrame, stat: str = "kills", view_name="player_cumulative_view"):
    df, stat_cols = initialize_cumulative_columns(df, stat=stat)
    
    for game_id in tqdm(df["game_id"], desc=f"Filling {stat}"):
        # Fetch the game data
        game = client.query_dataframe(
            "SELECT * FROM games_flatten WHERE game_id = %(game_id)s",
            params={"game_id": game_id}
        )
        if game.empty:
            continue
        
        begin_at = game["begin_at"].iloc[0]
        team_player = game.groupby("team_id")["player_id"].apply(lambda x: np.unique(x)).to_dict()
        
        
        stat_all = []
        for team_id, p_ids in team_player.items():
            stat_all.extend(sorted([client.query_dataframe(
                f"SELECT sum({stat}) AS value FROM {view_name} WHERE player_id = %(player_id)s AND begin_at < %(begin_at)s",
                params={"player_id": p_id, "begin_at": begin_at}
            )["value"].iloc[0]
            for p_id in p_ids]))
        
        df.loc[df["game_id"] == game_id, stat_cols] = stat_all
    
    return df

# -----------------------------
# Usage
# -----------------------------
game_ids = client.query_dataframe("SELECT DISTINCT(game_id) FROM games_flatten ORDER BY begin_at")["game_id"].values
game_ids_train, game_ids_test = game_ids[:-100], game_ids[-100:]

df_train = fetch_games_by_ids(client, game_ids_train)
df_test  = fetch_games_by_ids(client, game_ids_test)

df_train = fill_cumulative_stat(client, df_train, stat="kills")
df_test  = fill_cumulative_stat(client, df_test, stat="kills")


Filling kills:   0%|          | 0/37031 [00:00<?, ?it/s]

ValueError: Must have equal len keys and value when setting with an iterable

In [58]:
df_train

,game_id,timestamp,map_id,league_id,serie_id,tournament_id,tier_id,t1_id,t2_id,p1_id,...,p1_kills_sum,p2_kills_sum,p3_kills_sum,p4_kills_sum,p5_kills_sum,p6_kills_sum,p7_kills_sum,p8_kills_sum,p9_kills_sum,p10_kills_sum
0,1744,2016-04-13 16:30:00,8,4569,1544,1583,1,3216,3224,17539,...,0,0,0,0,0,0,0,0,0,0
1,2365,2016-06-11 03:01:00,8,4154,1559,1714,0,3209,3224,17507,...,0,0,0,0,0,0,0,0,0,0
2,1287,2016-09-08 13:30:00,8,4165,1591,1574,0,3209,3216,17507,...,0,0,0,0,0,0,0,0,0,0
3,2011,2016-12-10 00:40:00,4,4153,1552,1567,0,3222,3245,17513,...,0,0,0,0,0,0,0,0,0,0
4,4436,2017-03-01 21:00:00,3,4161,1583,1642,0,3212,3245,17522,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37026,70114,2022-04-13 10:15:39,2,4163,4541,7883,2,126709,129501,20569,...,0,0,0,0,0,0,0,0,0,0
37027,70112,2022-04-13 10:18:37,8,4163,4541,7883,2,125802,126377,20678,...,0,0,0,0,0,0,0,0,0,0
37028,70113,2022-04-13 11:29:14,2,4163,4541,7883,2,125802,126377,20678,...,0,0,0,0,0,0,0,0,0,0
37029,70115,2022-04-13 11:30:45,7,4163,4541,7883,2,126709,129501,20569,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_train.loc[df_train["game_id"]]

,game_id,timestamp,map_id,league_id,serie_id,tournament_id,tier_id,t1_id,t2_id,p1_id,p2_id,p3_id,p4_id,p5_id,p6_id,p7_id,p8_id,p9_id,p10_id,team1_win
0,1744,2016-04-13 16:30:00,8,4569,1544,1583,1,3216,3224,17539,17540,17542,17570,17602,17529,17576,17578,17579,17588,1
1,2365,2016-06-11 03:01:00,8,4154,1559,1714,0,3209,3224,17507,17508,17509,17510,17524,17529,17576,17578,17579,17655,0
2,1287,2016-09-08 13:30:00,8,4165,1591,1574,0,3209,3216,17507,17508,17509,17510,17524,17525,17539,17540,17541,17542,1
3,2011,2016-12-10 00:40:00,4,4153,1552,1567,0,3222,3245,17513,17515,17516,17568,17708,17501,17581,17658,17664,17665,1
4,4436,2017-03-01 21:00:00,3,4161,1583,1642,0,3212,3245,17522,17523,17524,17565,17569,17501,17543,17581,17664,17665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37026,70114,2022-04-13 10:15:39,2,4163,4541,7883,2,126709,129501,20569,25589,27555,28854,34000,17499,17747,21433,21439,25439,0
37027,70112,2022-04-13 10:18:37,8,4163,4541,7883,2,125802,126377,20678,20684,20692,23776,29284,17497,17498,17501,17543,19666,0
37028,70113,2022-04-13 11:29:14,2,4163,4541,7883,2,125802,126377,20678,20684,20692,23776,29284,17497,17498,17501,17543,19666,0
37029,70115,2022-04-13 11:30:45,7,4163,4541,7883,2,126709,129501,20569,25589,27555,28854,34000,17499,17747,21433,21439,25439,1


ServerException: Code: 62.
DB::Exception: Syntax error: failed at position 88 ('14'): 14:09:41 . Expected one of: token, DoubleColon, OR, AND, IS NOT DISTINCT FROM, IS NULL, IS NOT NULL, BETWEEN, NOT BETWEEN, LIKE, ILIKE, NOT LIKE, NOT ILIKE, REGEXP, IN, NOT IN, GLOBAL IN, GLOBAL NOT IN, MOD, DIV, alias, AS, GROUP BY, WITH, HAVING, WINDOW, ORDER BY, LIMIT, OFFSET, FETCH, SETTINGS, UNION, EXCEPT, INTERSECT, INTO OUTFILE, FORMAT, end of query. Stack trace:

0. DB::Exception::Exception(DB::Exception::MessageMasked&&, int, bool) @ 0x000000000cdd0af7 in /usr/bin/clickhouse
1. DB::Exception::createDeprecated(String const&, int, bool) @ 0x00000000079b862d in /usr/bin/clickhouse
2. DB::parseQueryAndMovePosition(DB::IParser&, char const*&, char const*, String const&, bool, unsigned long, unsigned long) @ 0x00000000142b13fc in /usr/bin/clickhouse
3. DB::executeQueryImpl(char const*, char const*, std::shared_ptr<DB::Context>, DB::QueryFlags, DB::QueryProcessingStage::Enum, DB::ReadBuffer*) @ 0x00

array([ 1744,  2365,  1287, ..., 70490, 70489, 70491], shape=(37131,))

In [ ]:
def fetch_games_by_ids(client: Client, game_ids, clickhouse_db="cs2_db"):
    if not game_ids:
        log.warning("Список game_ids пуст. Возвращаю пустые массивы.")
        return np.array([]), np.array([])
    log.info(f"Получение данных для {len(game_ids)} игр из ClickHouse...")
    ids_str = ",".join(str(i) for i in game_ids)
    query = f"""
    SELECT
        g.game_id,
        any(g.begin_at) AS timestamp,
        any(g.map_id) AS map_id,
        any(g.league_id) AS league_id,
        any(g.serie_id) AS serie_id,
        any(g.tournament_id) AS tournament_id,
        any(g.tier_id) AS tier_id,
        t.teams[1] AS t1_id,
        t.teams[2] AS t2_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 1) AS p1_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 2) AS p2_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 3) AS p3_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 4) AS p4_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[1]))), 1, 5), 5) AS p5_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 1) AS p6_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 2) AS p7_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 3) AS p8_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 4) AS p9_id,
        arrayElement(arraySlice(arraySort(arrayDistinct(groupArrayIf(player_id, team_id = t.teams[2]))), 1, 5), 5) AS p10_id,
        sumIf(g.round_win, g.team_id = t.teams[1]) > sumIf(g.round_win, g.team_id = t.teams[2]) AS team1_win
    FROM {clickhouse_db}.games_flatten AS g
    INNER JOIN (
        SELECT
            game_id,
            arraySort(arrayDistinct(groupArray(team_id))) AS teams
        FROM {clickhouse_db}.games_flatten
        WHERE game_id IN ({ids_str})
        GROUP BY game_id
    ) AS t USING (game_id)
    GROUP BY g.game_id, t.teams
    ORDER BY timestamp ASC
    """
    result = client.execute(query, with_column_types=True)
    if not result:
        log.warning("Запрос вернул пустой результат.")
        return np.array([]), np.array([])
    rows = [list(r) for r in result[0]]
    for row in rows:
        if hasattr(row[1], "timestamp"):
            row[1] = row[1].timestamp()
    X = np.array([r[1:-1] for r in rows], dtype=float)
    y = np.array([r[-1] for r in rows], dtype=int)
    log.info(f"Получено {len(X)} строк с {X.shape[1]} признаками.")
    return X, y

,uniqExact_game_id_
0,37131
